In [1]:
# Filtering the war
import warnings
warnings.filterwarnings('ignore')

In [2]:
#importing dependencies
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import shutil
import glob

In [3]:
#count the number of images in each set 0---> YES 1-->NO
ROOT_DIR="D:\Internship Code Clause\Task1 Brain Tumour Detection\Dataset"

number_of_images={}
for dir in os.listdir(ROOT_DIR):
    number_of_images[dir]=len(os.listdir(os.path.join(ROOT_DIR,dir)))

In [4]:
number_of_images.items()

dict_items([('Brain Tumor', 2433), ('Healthy', 2087)])

In [5]:
# we will split the data such that 70% train data and 15% validataion and 15% test data

In [6]:
# Function to create the train,test,validation sets.
def foldercreator(path,split):
    if not os.path.exists("./"+path):
        os.mkdir("./"+path)
        for dir in os.listdir(ROOT_DIR):
            os.makedirs("./"+path+"./"+dir)
            for img in np.random.choice(a=os.listdir(os.path.join(ROOT_DIR,dir)),size=(math.floor(split*number_of_images[dir])-5),replace=False):
                o=os.path.join(ROOT_DIR,dir,img)# path
                D=os.path.join("./"+path,dir)
                shutil.copy(o,D)
                os.remove(o)

    else:
        print("The folder exsist")

In [7]:
foldercreator("train",0.7)
foldercreator("test",0.15)
foldercreator("validation",0.15)